<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импортируем-модули" data-toc-modified-id="Импортируем-модули-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импортируем модули</a></span></li><li><span><a href="#Устанавливаем-подключение" data-toc-modified-id="Устанавливаем-подключение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Устанавливаем подключение</a></span></li><li><span><a href="#Набор-переменных-для-расчета" data-toc-modified-id="Набор-переменных-для-расчета-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Набор переменных для расчета</a></span></li><li><span><a href="#Планируемое-кол-во-флота-с-учетом-ремонта" data-toc-modified-id="Планируемое-кол-во-флота-с-учетом-ремонта-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Планируемое кол-во флота с учетом ремонта</a></span></li><li><span><a href="#Загружаем-необходимые-слои-из-Авроры" data-toc-modified-id="Загружаем-необходимые-слои-из-Авроры-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Загружаем необходимые слои из Авроры</a></span></li><li><span><a href="#Объединяем-зоны-запрета-и-плохое-землепользование" data-toc-modified-id="Объединяем-зоны-запрета-и-плохое-землепользование-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Объединяем зоны запрета и плохое землепользование</a></span><ul class="toc-item"><li><span><a href="#Сущпол-положение" data-toc-modified-id="Сущпол-положение-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Сущпол положение</a></span></li><li><span><a href="#Проектное-предложение" data-toc-modified-id="Проектное-предложение-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Проектное предложение</a></span></li></ul></li><li><span><a href="#Устанавливаем-систему-координат-для-города" data-toc-modified-id="Устанавливаем-систему-координат-для-города-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Устанавливаем систему координат для города</a></span></li><li><span><a href="#Расчитываем-полезную-площадь-зоны-парковки" data-toc-modified-id="Расчитываем-полезную-площадь-зоны-парковки-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Расчитываем полезную площадь зоны парковки</a></span></li><li><span><a href="#Расчитываем-плотность-самокатов" data-toc-modified-id="Расчитываем-плотность-самокатов-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Расчитываем плотность самокатов</a></span></li><li><span><a href="#Сводная-таблица-с-плотностью-самокатов-для-полезной-зоны-парковки" data-toc-modified-id="Сводная-таблица-с-плотностью-самокатов-для-полезной-зоны-парковки-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Сводная таблица с плотностью самокатов для полезной зоны парковки</a></span></li><li><span><a href="#Пересечение-зоны-парковок-с-предложенными-операционными-зонами" data-toc-modified-id="Пересечение-зоны-парковок-с-предложенными-операционными-зонами-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Пересечение зоны парковок с предложенными операционными зонами</a></span></li><li><span><a href="#Кол-во-парковок-и-самокатов-по-операционным-зонам" data-toc-modified-id="Кол-во-парковок-и-самокатов-по-операционным-зонам-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Кол-во парковок и самокатов по операционным зонам</a></span></li></ul></div>

## Импортируем модули

In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt, wkb
from shapely.geometry import MultiPolygon
from sqlalchemy import create_engine
import folium

## Устанавливаем подключение

In [2]:
driver = 'postgresql+psycopg2://'
user = 'georgy_ovnanyan'
password = ''
hostname = 'whoosh-prod-aurora.cluster-cqlsvpdknncb.eu-west-1.rds.amazonaws.com:5432'
db_name = 'db-whoosh'
engine = create_engine('{driver}{user}:{password}@{hostname}/{db_name}'.format(
                                                                                driver = driver,
                                                                                user = user,
                                                                                password = password,
                                                                                hostname = hostname,
                                                                                db_name = db_name)
)

## Набор переменных для расчета

In [3]:
scooters_coef = 0.9
current_scooters_all = 1617
forecast_scooters_all = 2177
crs_project = 32643
region_id = 80
location = [54.989347, 73.368221]

In [5]:
op_zones_proposed = gpd.read_file(r'/Users/georgy/qgis/Ad-hoc//Омск/Слои/Предлагаемые опер зоны.shp')
op_zones_proposed = op_zones_proposed.set_crs(4326)
op_zones_proposed = op_zones_proposed[['name', 'region_id', 'geometry']]
op_zones_proposed.head()

,id,geometry
0,1,"POLYGON ((86.22573 55.42884, 86.23103 55.43001..."


## Планируемое кол-во флота с учетом ремонта

In [47]:
current_scooters = current_scooters_all * scooters_coef
forecast_scooters = forecast_scooters_all * scooters_coef

## Загружаем необходимые слои из Авроры

In [48]:
parkings = gpd.GeoDataFrame.from_postgis('''select id, internal_id, region_id, address, coordinate_geom
                                            from public.parkings
                                            where region_id = {}
                                                and location_permission = 'PERMITTED'
                                                and status = 'VERIFIED'
                                         '''.format(region_id), con=engine, 
                                              geom_col='coordinate_geom', 
                                              crs='4326')

area = gpd.GeoDataFrame.from_postgis('''select name, region_id, borders_geom
                                        from public.area a
                                        where a.type = 'PERMITTED' 
                                              and a.status ='ACTIVE'
                                              and a.region_id = {}
                                     '''.format(region_id), con=engine, 
                                          geom_col='borders_geom', 
                                          crs='4326')

area_proposed = gpd.GeoDataFrame.from_postgis('''select name, region_id, st_makevalid(st_union(borders_geom)) as borders_geom
                                                 from product.permitted_proposed pp
                                                 where region_id = {}
                                                       and "type" = 'PERMITTED'
                                                       and status = 'PROPOSED'  
                                                 group by 1,2
                                             '''.format(region_id), con=engine, 
                                                  geom_col='borders_geom', 
                                                  crs='4326')

forbidden_current = gpd.GeoDataFrame.from_postgis('''select ag.region_id,
                                             st_makevalid(st_union(st_intersection(ST_MakeValid(st_transform(a.borders_geom,4326)), ST_MakeValid(st_transform(ag.borders_geom,4326))))) as geom
                                             from (select region_id, borders_geom
                                                   from public.area
                                                   where "type" = 'PERMITTED' 
                                                          and status ='ACTIVE'
                                                          and region_id = {}) as ag
                                             join (select region_id, borders_geom
                                                   from public.area 
                                                   where "type" = 'FORBIDDEN'
                                                            and status = 'ACTIVE'
                                                            and is_fictitious is false
                                                            and region_id = {}) as a on st_intersects(a.borders_geom, ag.borders_geom)
                                              group by ag.region_id
                                             '''.format(region_id, region_id), con=engine, 
                                                  geom_col='geom', 
                                                  crs='4326')

forbidden_proposed = gpd.GeoDataFrame.from_postgis('''select pp.region_id,
                                             st_makevalid(st_union(st_intersection(ST_MakeValid(st_transform(a.borders_geom,4326)), ST_MakeValid(st_transform(pp.borders_geom,4326))))) as geom
                                             from (select region_id, borders_geom
                                                   from product.permitted_proposed
                                                   where "type" = 'PERMITTED' 
                                                          and region_id = {}
                                                          and status = 'PROPOSED') as pp
                                             join (select region_id, borders_geom
                                                   from public.area 
                                                   where "type" = 'FORBIDDEN'
                                                            and status = 'ACTIVE'
                                                            and is_fictitious is false
                                                            and region_id = {}) as a on st_intersects(a.borders_geom, pp.borders_geom)
                                              group by pp.region_id
                                             '''.format(region_id, region_id), con=engine, 
                                                  geom_col='geom', 
                                                  crs='4326')

bad_landuse_proposed = gpd.GeoDataFrame.from_postgis('''select pp.region_id,
                                                      lbg."type",
                                                      st_makevalid(st_union(st_intersection(lbg.geom, pp.borders_geom))) as geom
                                               from martin.landuse_boundaries_geom as lbg
                                               join product.permitted_proposed  as pp on st_intersects(pp.borders_geom, lbg.geom)
                                               where pp.region_id = {} 
                                                     and pp.status = 'PROPOSED'  
                                                     and pp."type" = 'PERMITTED'
                                                     and lbg."type" = 'bad'
                                               group by 1,2
                                             '''.format(region_id), con=engine, 
                                                  geom_col='geom', 
                                                  crs='4326')

bad_landuse_current = gpd.GeoDataFrame.from_postgis('''select a.region_id,
                                                          lbg."type",
                                                          st_makevalid(st_union(st_intersection(lbg.geom, a.borders_geom))) as geom
                                                   from martin.landuse_boundaries_geom as lbg
                                                   join public.area as a on st_intersects(a.borders_geom, lbg.geom)
                                                   where a.type = 'PERMITTED' 
                                                          and a.status ='ACTIVE'
                                                          and a.region_id = {} 
                                                          and lbg."type" = 'bad'
                                                   group by 1,2
                                             '''.format(region_id), con=engine, 
                                                  geom_col='geom', 
                                                  crs='4326')

In [49]:
# небольшая проверка дублирования слоев
for i in [area, area_proposed, forbidden_current, forbidden_proposed, bad_landuse_proposed, bad_landuse_current]:
    if len(i) == 1:
        print('ok')
    else: 
        print('дублирование')

ok
ok
ok
ok
ok
ok


In [50]:
# area = area.dissolve(by='region_id', as_index=False)
# area_proposed = area_proposed.dissolve(by='region_id', as_index=False)
# area['name'] = 'Томск'

## Объединяем зоны запрета и плохое землепользование

### Сущпол положение

In [51]:
bad_area_current = gpd.overlay(bad_landuse_current, forbidden_current, how='union')
bad_area_current = bad_area_current.drop(['region_id_2'], axis=1).rename(columns={'region_id_1':'region_id'})
bad_area_current['region_id'] = bad_area_current['region_id'].fillna(method='ffill')
bad_area_current = bad_area_current[['region_id', 'geometry']].dissolve('region_id').reset_index()
bad_area_current

,region_id,geometry
0,80.0,"MULTIPOLYGON (((73.27735 54.96917, 73.27735 54..."


### Проектное предложение

In [52]:
bad_area_proposed = gpd.overlay(bad_landuse_proposed, forbidden_proposed, how='union')
bad_area_proposed = bad_area_proposed.drop(['region_id_2'], axis=1).rename(columns={'region_id_1':'region_id'})
bad_area_proposed['region_id'] = bad_area_proposed['region_id'].fillna(method='ffill')
bad_area_proposed = bad_area_proposed[['region_id', 'geometry']].dissolve('region_id').reset_index()
bad_area_proposed

,region_id,geometry
0,80.0,"MULTIPOLYGON (((73.27735 54.96917, 73.27735 54..."


## Устанавливаем систему координат для города

In [53]:
parkings = parkings.to_crs(crs_project)
area = area.to_crs(crs_project)
area_proposed = area_proposed.to_crs(crs_project)
op_zones_proposed = op_zones_proposed.to_crs(crs_project)
bad_area_proposed = bad_area_proposed.to_crs(crs_project)
bad_area_current = bad_area_current.to_crs(crs_project)

In [54]:
m = folium.Map(location=location, zoom_start=11, tiles="CartoDB positron")
folium.GeoJson(area['borders_geom'], style_function=lambda feature:{"fillColor": "blue"}).add_to(m)
# folium.GeoJson(bad_area_current['geometry'], style_function=lambda feature:{"fillColor": "red"}).add_to(m)
folium.GeoJson(area_proposed['borders_geom'], style_function=lambda feature:{"fillColor": "red"}).add_to(m)
# folium.GeoJson(bad_area_proposed['geometry'], style_function=lambda feature:{"fillColor": "red"}).add_to(m)
# folium.GeoJson(parkings['coordinate_geom'],
#                    marker=folium.Circle(
#                    radius=4,
#                    fill_color="orange",
#                    fill_opacity=0.4,
#                    color="black",
#                    weight=1)).add_to(m)
m

## Расчитываем полезную площадь зоны парковки

In [55]:
current_area_square = round(area.geometry[0].area/1000000 - bad_area_current.geometry[0].area/1000000,2)
proposed_area_square = round(area_proposed.geometry[0].area/1000000 - bad_area_proposed.geometry[0].area/1000000,2)
print(current_area_square)
print(proposed_area_square)

42.47
42.47


## Расчитываем плотность самокатов

In [56]:
current_scooter_density = round(current_scooters / current_area_square,1)
forecast_scooter_density = round(forecast_scooters / proposed_area_square,1)
print(current_scooter_density)
print(forecast_scooter_density)

34.3
46.1


## Сводная таблица с плотностью самокатов для полезной зоны парковки

In [57]:
pd.DataFrame({'year':[2023, 2024], 
              'area':[current_area_square, proposed_area_square],
              'scooters_all':[current_scooters_all, forecast_scooters_all],
              'density':[current_scooter_density, forecast_scooter_density]
              })

,year,area,scooters_all,density
0,2023,42.47,1617,34.3
1,2024,42.47,2177,46.1


## Пересечение зоны парковок с предложенными операционными зонами

In [58]:
permitted_op_zones = gpd.overlay(area_proposed, op_zones_proposed, how = 'intersection')
permitted_op_zones.columns = ['name_city', 'region_id_1', 'name_zone', 'region_id_2', 'geometry']
permitted_op_zones = gpd.overlay(permitted_op_zones, bad_area_proposed, how='difference')
permitted_op_zones['area'] = round(permitted_op_zones.geometry.area/1000000, 1)
permitted_op_zones.head()

,name_city,region_id_1,name_zone,region_id_2,geometry,area
0,Омск,80,Омск - ИртышскаяНабережная,80.0,"POLYGON ((395667.141 6089637.894, 395684.796 6...",0.3
1,Омск,80,Омск Центр Юг,80.0,"MULTIPOLYGON (((395886.075 6093349.319, 395885...",6.0
2,Омск,80,Омск Северные Восток,80.0,"MULTIPOLYGON (((394959.088 6097874.170, 394976...",4.8
3,Омск,80,Омск Левбер Север,80.0,"MULTIPOLYGON (((390464.122 6094107.598, 390468...",4.6
4,Омск,80,Омск - ЗеленыйОстров,80.0,"MULTIPOLYGON (((393537.152 6096587.771, 393540...",0.8


In [59]:
j = folium.Map(location=location, zoom_start=11, tiles="CartoDB positron")
folium.GeoJson(permitted_op_zones['geometry']).add_to(j)
j

## Кол-во парковок и самокатов по операционным зонам

In [60]:
parkings_op = gpd.sjoin(permitted_op_zones, parkings, how = 'left') 
parkings_op = parkings_op.groupby(
    ['name_zone', 'region_id', 'name_city', 'geometry', 'area']
)['id'].count().reset_index()
parkings_op = parkings_op.rename(columns = {'id':'cnt_parkings'}) 
parkings_op['parking_density'] = round(parkings_op['cnt_parkings'] / parkings_op['area'], 1)
parkings_op['delta_parkings'] = parkings_op['cnt_parkings'] - round(parkings_op['area'] * 19, 1)
parkings_op

,name_zone,region_id,name_city,geometry,area,cnt_parkings,parking_density,delta_parkings
0,Омск - ЗеленыйОстров,80,Омск,"MULTIPOLYGON (((393537.152 6096587.771, 393540...",0.8,18,22.5,2.8
1,Омск - ИртышскаяНабережная,80,Омск,"POLYGON ((395667.141 6089637.894, 395684.796 6...",0.3,31,103.3,25.3
2,Омск Левбер Север,80,Омск,"MULTIPOLYGON (((390464.122 6094107.598, 390468...",4.6,106,23.0,18.6
3,Омск Левбер Юг,80,Омск,"MULTIPOLYGON (((394604.882 6094993.021, 394598...",5.0,83,16.6,-12.0
4,Омск Нефтяники,80,Омск,"MULTIPOLYGON (((391468.979 6098600.684, 391463...",8.0,175,21.9,23.0
5,Омск Северные Восток,80,Омск,"MULTIPOLYGON (((394959.088 6097874.170, 394976...",4.8,71,14.8,-20.2
6,Омск Северные Запад,80,Омск,"MULTIPOLYGON (((393118.163 6097563.753, 393011...",2.6,25,9.6,-24.4
7,Омск Центр Север,80,Омск,"MULTIPOLYGON (((395181.390 6094257.857, 394998...",3.5,112,32.0,45.5
8,Омск Центр Юг,80,Омск,"MULTIPOLYGON (((395886.075 6093349.319, 395885...",6.0,115,19.2,1.0
9,Омск Юг Восток,80,Омск,"MULTIPOLYGON (((398535.622 6091188.480, 398535...",2.7,44,16.3,-7.3


In [61]:
result = permitted_op_zones[['name_zone', 'geometry', 'area']].merge(
    parkings_op.drop(['geometry', 'area'], axis=1), 
    on='name_zone', 
    how='left'
)
result

,name_zone,geometry,area,region_id,name_city,cnt_parkings,parking_density,delta_parkings
0,Омск - ИртышскаяНабережная,"POLYGON ((395667.141 6089637.894, 395684.796 6...",0.3,80,Омск,31,103.3,25.3
1,Омск Центр Юг,"MULTIPOLYGON (((395886.075 6093349.319, 395885...",6.0,80,Омск,115,19.2,1.0
2,Омск Северные Восток,"MULTIPOLYGON (((394959.088 6097874.170, 394976...",4.8,80,Омск,71,14.8,-20.2
3,Омск Левбер Север,"MULTIPOLYGON (((390464.122 6094107.598, 390468...",4.6,80,Омск,106,23.0,18.6
4,Омск - ЗеленыйОстров,"MULTIPOLYGON (((393537.152 6096587.771, 393540...",0.8,80,Омск,18,22.5,2.8
5,Омск Юг Восток,"MULTIPOLYGON (((398535.622 6091188.480, 398535...",2.7,80,Омск,44,16.3,-7.3
6,Омск Нефтяники,"MULTIPOLYGON (((391468.979 6098600.684, 391463...",8.0,80,Омск,175,21.9,23.0
7,Омск Левбер Юг,"MULTIPOLYGON (((394604.882 6094993.021, 394598...",5.0,80,Омск,83,16.6,-12.0
8,Омск Центр Север,"MULTIPOLYGON (((395181.390 6094257.857, 394998...",3.5,80,Омск,112,32.0,45.5
9,Омск Северные Запад,"MULTIPOLYGON (((393118.163 6097563.753, 393011...",2.6,80,Омск,25,9.6,-24.4


In [62]:
result['scooters'] = result['area'] * 40
result['per_scooters'] = result['scooters'] / result['scooters'].sum()
print(result['scooters'].sum())
result

1704.0


,name_zone,geometry,area,region_id,name_city,cnt_parkings,parking_density,delta_parkings,scooters,per_scooters
0,Омск - ИртышскаяНабережная,"POLYGON ((395667.141 6089637.894, 395684.796 6...",0.3,80,Омск,31,103.3,25.3,12.0,0.007042
1,Омск Центр Юг,"MULTIPOLYGON (((395886.075 6093349.319, 395885...",6.0,80,Омск,115,19.2,1.0,240.0,0.140845
2,Омск Северные Восток,"MULTIPOLYGON (((394959.088 6097874.170, 394976...",4.8,80,Омск,71,14.8,-20.2,192.0,0.112676
3,Омск Левбер Север,"MULTIPOLYGON (((390464.122 6094107.598, 390468...",4.6,80,Омск,106,23.0,18.6,184.0,0.107981
4,Омск - ЗеленыйОстров,"MULTIPOLYGON (((393537.152 6096587.771, 393540...",0.8,80,Омск,18,22.5,2.8,32.0,0.018779
5,Омск Юг Восток,"MULTIPOLYGON (((398535.622 6091188.480, 398535...",2.7,80,Омск,44,16.3,-7.3,108.0,0.063380
6,Омск Нефтяники,"MULTIPOLYGON (((391468.979 6098600.684, 391463...",8.0,80,Омск,175,21.9,23.0,320.0,0.187793
7,Омск Левбер Юг,"MULTIPOLYGON (((394604.882 6094993.021, 394598...",5.0,80,Омск,83,16.6,-12.0,200.0,0.117371
8,Омск Центр Север,"MULTIPOLYGON (((395181.390 6094257.857, 394998...",3.5,80,Омск,112,32.0,45.5,140.0,0.082160
9,Омск Северные Запад,"MULTIPOLYGON (((393118.163 6097563.753, 393011...",2.6,80,Омск,25,9.6,-24.4,104.0,0.061033


In [63]:
result.to_excel(r'/Users/georgy/qgis/Подготовка к сезону/Омск/распределние флота и парковок.xlsx', 
                index=False)